# Securing access to External Tables / Files with Unity Catalog

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/external/uc-external-location-global.png?raw=true" style="float:right; margin-left:10px" width="600"/>

By default, Unity Catalog will create managed tables in your primary storage, providing a secured table access for all your users.

In addition to these managed tables, you can manage access to External tables and files, located in another cloud storage (S3/ADLS/GCS). 

This give you capabilities to ensure a full data governance, storing your main tables in the managed catalog/storage while ensuring secure access for for specific cloud storage.

<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=governance&org_id=4214571749987147&notebook=%2FAzure-Securing-data-on-external-locations&demo_name=uc-02-external-location&event=VIEW&path=%2F_dbdemos%2Fgovernance%2Fuc-02-external-location%2FAzure-Securing-data-on-external-locations&version=1">


## Working with External Locations

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/external/azure-uc-external-location.png?raw=true" style="float:right; margin-left:10px" width="800"/>


Accessing external cloud storage is easily done using `External locations`.

This can be done using 3 simple SQL command:


1. First, create a Storage credential. It'll reference the Managed Identity or Service Principal required to access your cloud storage
1. Create an External location using your Storage credential. It can be any cloud location (a sub folder)
1. Finally, Grant permissions to your users to access this Storage Credential

## 1/ Create the STORAGE CREDENTIAL

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/external/azure-uc-external-location-1.png?raw=true" style="float:right; margin-left:10px" width="700px"/>

The first step is to create the `STORAGE CREDENTIAL`.

To do that, we'll use Databricks Unity Catalog UI:

1. Open the Data Explorer in DBSQL
1. Select the "Storage Credential" menu
1. Click on "Create Credential"
1. Since it is recommended to use a Managed Identity, fill out the fields with your credential information: Storage credential name, Access connector ID, and optionally your User assigned managed identity ID.


<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/external/azure-storage-credential.png?raw=true" width="400"/>

In [0]:
%sql
-- For our demo, let's make sure all users can alter this storage credential:
-- ALTER STORAGE CREDENTIAL `field_demos_credential`  OWNER TO `account users`;

In [0]:
%sql
SHOW STORAGE CREDENTIALS 

In [0]:
%sql
DESCRIBE STORAGE CREDENTIAL `field_demos_credential`

## 2/ Create the EXTERNAL LOCATION

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/external/azure-uc-external-location-2.png?raw=true" style="float:right; margin-left:10px" width="700px"/>


We'll then create our `EXTERNAL LOCATION` using the following path:<br/>
`abfss://deltalake@oneenvadls.dfs.core.windows.net/demofieldeng/external_location/`

Note that you need to be Account Admin to do that, it'll fail with a permission error if you are not. But don't worry, the external location has been created for you.

You can also update your location using SQL operations:
<br/>
```ALTER EXTERNAL LOCATION `xxxx`  RENAME TO `yyyy`; ```<br/>
```DROP EXTERNAL LOCATION IF EXISTS `xxxx`; ```

In [0]:
%sql
-- Replace this with your own path
CREATE EXTERNAL LOCATION IF NOT EXISTS `field_demos_external_location`
  URL 'abfss://deltalake@oneenvadls.dfs.core.windows.net/demofieldeng/external_location/' 
  WITH (CREDENTIAL `field_demos_credential`)
  COMMENT 'External Location for demos' ;

-- let's make everyone owner for the demo to be able to change the permissions easily. DO NOT do that for real usage.
ALTER EXTERNAL LOCATION `field_demos_external_location`  OWNER TO `account users`;

In [0]:
%sql
SHOW EXTERNAL LOCATIONS

In [0]:
%sql
DESCRIBE EXTERNAL LOCATION `field_demos_external_location`;

## 3/ GRANT permissions on the external location

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/external/azure-uc-external-location-3.png?raw=true" style="float:right; margin-left:10px" width="700px"/>

All we have to do is now GRANT permission to our users or group of users. In our demo we'll grant access to all our users using `account users`

We can set multiple permissions:

1. READ FILES to be able to access the data
1. WRITE FILES to be able to write data
1. CREATE TABLE to create external table using this location

To revoke your permissions, you can use ```REVOKE WRITE FILES ON EXTERNAL LOCATION `field_demos_external_location` FROM `account users`;```

In [0]:
%sql
GRANT READ FILES, WRITE FILES ON EXTERNAL LOCATION `field_demos_external_location` TO `account users`;

## Accessing the data

That's all we have to do! Our users can now access the folder in SQL or python:

In [0]:
%sql
LIST 'abfss://deltalake@oneenvadls.dfs.core.windows.net/demofieldeng/external_location/'

we can also write data using SQL or Python API:

In [0]:
df = spark.createDataFrame([("UC", "is awesome"), ("Delta Sharing", "is magic")])
df.write.mode('overwrite').format('csv').save('abfss://deltalake@oneenvadls.dfs.core.windows.net/demofieldeng/external_location/test_write')

In [0]:
spark.read.csv('abfss://deltalake@oneenvadls.dfs.core.windows.net/demofieldeng/external_location/test_write').display()


Setting the Permissions can also be done using the Data Explorer UI:

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/external/uc-external-location-cred2.png?raw=true" width="400" >

*Note: because we have set all users to OWNER for the demo, all users have full READ/WRITE permissions as OWNER (even without the GRANT). In a real setup, a single admin would be the OWNER, granting specific access to group of users or specific users.*

## Conclusion

With Unity Catalog, you can easily secure access to external locations and grant access based on users/groups.

This let you operate security at scale, cross workspace, and be ready to build data mesh setups.